Now its your time to build a neural network. Build one that takes binary numbers from 000 to 111 and classifies them as ≥ 5 or not

In [ ]:
import sys, re
import numpy as np
import tensorflow as tf

In [ ]:
import shutil
shutil.rmtree("examplemodel")

This is what the input/output looks like:

In [ ]:
train_data = np.asarray([[0,0,0],[0,0,1],[0,1,0],[0,1,1],[1,0,0],[1,0,1],[1,1,0],[1,1,1]], dtype=np.float32)
train_labels= np.asarray([0      ,0      ,0      ,0      ,0      ,1      ,1      ,1], dtype=np.int32)
print(str(train_data.shape))
print(str(train_labels.shape))

Your turn: modify this code as needed:

In [ ]:
def model_function(features, labels, mode):
    input_layer = tf.reshape(features["x"], [-1, 1])
    hidden = tf.layers.dense(inputs=input_layer, units=1)
    logits = tf.layers.dense(inputs=hidden, units=2)
    predictions = {
        "classes": tf.argmax(input=logits, axis=1),
        "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
    tf.summary.scalar("loss", loss)

    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {"accuracy": tf.metrics.accuracy(labels=labels, predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)


Create the classifier and train

In [ ]:
estimator = tf.estimator.Estimator(model_fn=model_function, model_dir="examplemodel")
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": train_data},
    y=train_labels,
    batch_size=4,
    num_epochs=None,
    shuffle=True)
estimator.train( input_fn=train_input_fn, steps=1000 )

Evaluate model...

In [ ]:
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
  x={"x":train_data},
  y=train_labels,
  num_epochs=1,
  shuffle=False)
eval_results = estimator.evaluate(input_fn=eval_input_fn)
print(eval_results)

In [ ]:
test_data = [1, 1, 0]
test_data = np.asarray(test_data, dtype=np.float32)
eval_input_fn = tf.estimator.inputs.numpy_input_fn( x={"x":test_data}, num_epochs=1, shuffle=False )
eval_results = estimator.predict(input_fn=eval_input_fn)
result = eval_results.__next__()
if "1" in str(result["classes"]):
    print(str(test_data)+" is greater or equal to 5")
else:
    print(str(test_data)+" is less than 5")


Now how do you think the model did the classification?